In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('../data/final/wrangled/normalized.csv', index_col='year')
x_vars = [c for c in df.columns if c != 'instability']

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler

# Standardize the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Define the size of the encoding
encoding_dim = 8  # This can be adjusted

# Input layer
input_df = Input(shape=(df_scaled.shape[1],))

# Encoding layers
encoded = Dense(encoding_dim, activation='relu')(input_df)

# Decoding layers
decoded = Dense(df_scaled.shape[1], activation='sigmoid')(encoded)

# Autoencoder model
autoencoder = Model(input_df, decoded)

# Encoder model
encoder = Model(input_df, encoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
autoencoder.fit(df_scaled, df_scaled, epochs=50, batch_size=16, shuffle=True, validation_split=0.2)

# Encode the data
encoded_df = encoder.predict(df_scaled)